In [1]:
import graphlab

graphlab.canvas.set_target('ipynb')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1543135821.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1543135821.log


This non-commercial license of GraphLab Create for academic use is assigned to gaurav.agrawal@zs.com and will expire on October 05, 2019.


In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [5]:
model_all = graphlab.linear_regression.create(sales, target = 'price', features=all_features,
                                             validation_set = None, l1_penalty = 1e10, l2_penalty=0)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.763455     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.811507     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [13]:
model_all['coefficients'].sort('value', ascending=False)

name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
grade,None,842.068034898,None
sqft_living_sqrt,None,350.060553386,None
sqft_living,None,24.4207209824,None
sqft_above,None,20.0247224171,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


In [14]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [48]:
l1_penalty = np.logspace(1, 7, num=13)
print l1_penalty

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [49]:
v_frame = graphlab.SFrame()
for i in l1_penalty:
    model = graphlab.linear_regression.create(training, target='price', validation_set = None, verbose=False, l2_penalty=0,
                                                            l1_penalty=i, features=all_features)
    RSS_train = np.power(training['price']-model.predict(training),2).sum()
    RSS_valid = np.power(validation['price']-model.predict(validation),2).sum()
    NNZ = model['coefficients']['value'].nnz()
    #print (i, RSS_train, RSS_valid)
    v_sf = graphlab.SFrame({'l1':[i], 'RSS_valid': [RSS_valid], 'RSS_train': [RSS_train],'Non-Zero': [NNZ]})
    v_frame = v_frame.append(v_sf)

In [50]:
v_frame.sort('RSS_valid')

Non-Zero,RSS_train,RSS_valid,l1
18,7.30724649671e+14,6.25766285142e+14,10.0
18,7.30724649991e+14,6.25766285362e+14,31.6227766017
18,7.30724651003e+14,6.25766286058e+14,100.0
18,7.30724654205e+14,6.25766288257e+14,316.227766017
18,7.30724664327e+14,6.25766295212e+14,1000.0
18,7.30724696339e+14,6.25766317206e+14,3162.27766017
18,7.30724797572e+14,6.25766386761e+14,10000.0
18,7.3072511774e+14,6.25766606749e+14,31622.7766017
18,7.30726130614e+14,6.25767302792e+14,100000.0
18,7.30729337734e+14,6.25769507644e+14,316227.766017


In [51]:
model_l1_10 = graphlab.linear_regression.create(training, target = 'price', features=all_features,
                                             validation_set = None, l1_penalty = 10, l2_penalty=0)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.458576     | 5737572.318059     | 289374.784200 |

| 6         | 7        | 0.000003  | 0.489044     | 5655864.782013     | 285354.826809 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [52]:
model_l1_10['coefficients']['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In [53]:
max_nonzeros = 7

In [55]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

In [57]:
def ridge_regression(l1_penalty):
    v_frame = graphlab.SFrame()
    for i in l1_penalty:
        model = graphlab.linear_regression.create(training, target='price', validation_set = None, verbose=False, l2_penalty=0,
                                                                l1_penalty=i, features=all_features)
        RSS_train = np.power(training['price']-model.predict(training),2).sum()
        RSS_valid = np.power(validation['price']-model.predict(validation),2).sum()
        NNZ = model['coefficients']['value'].nnz()
        #print (i, RSS_train, RSS_valid)
        v_sf = graphlab.SFrame({'l1':[i], 'RSS_valid': [RSS_valid], 'RSS_train': [RSS_train],'Non-Zero': [NNZ]})
        v_frame = v_frame.append(v_sf)
    return(v_frame)

In [59]:
non_zero = ridge_regression(l1_penalty_values)

In [71]:
l1_penalty_max = non_zero[(non_zero['Non-Zero'] < max_nonzeros)]['l1'].min()

In [72]:
l1_penalty_min = non_zero[(non_zero['Non-Zero'] > max_nonzeros)]['l1'].max()

In [73]:
 print l1_penalty_min, l1_penalty_max

2976351441.63 3792690190.73


In [74]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [75]:
final = ridge_regression(l1_penalty_values)

In [78]:
final.sort('Non-Zero')

Non-Zero,RSS_train,RSS_valid,l1
6,1.24510305333e+15,1.08186759232e+15,3792690190.73
6,1.24041002772e+15,1.07763277558e+15,3749724993.41
6,1.23583082834e+15,1.07350454959e+15,3706759796.09
6,1.23134565524e+15,1.06946433543e+15,3663794598.77
6,1.22716519529e+15,1.0657076895e+15,3620829401.45
7,1.22175673046e+15,1.06079953176e+15,3577864204.13
7,1.21645802073e+15,1.05599273534e+15,3534899006.81
7,1.21111763204e+15,1.05114762561e+15,3491933809.48
7,1.20645885379e+15,1.04693748875e+15,3448968612.16
8,1.19720553554e+15,1.03855473594e+15,3363038217.52


In [80]:
final[(final['Non-Zero'] ==7)].sort('RSS_valid', ascending=True)

Non-Zero,RSS_train,RSS_valid,l1
7,1.20645885379e+15,1.04693748875e+15,3448968612.16
7,1.21111763204e+15,1.05114762561e+15,3491933809.48
7,1.21645802073e+15,1.05599273534e+15,3534899006.81
7,1.22175673046e+15,1.06079953176e+15,3577864204.13


In [81]:
final_model = graphlab.linear_regression.create(training, target = 'price', features=all_features,
                                             validation_set = None, l1_penalty = 3448968612.16, l2_penalty=0)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.466158     | 6622449.290291     | 361953.264500 |

| 6         | 7        | 0.000003  | 0.497920     | 6613330.420898     | 360099.555727 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [84]:
final_model.coefficients.sort('value', ascending=False)

name,index,value,stderr
(intercept),None,222253.192544,None
bathrooms,None,15873.9572593,None
grade,None,2899.42026975,None
sqft_living_sqrt,None,690.114773313,None
bedrooms,None,661.722717782,None
sqft_living,None,32.4102214513,None
sqft_above,None,30.0115753022,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


In [70]:
non_zero[(non_zero['Non-Zero'] >7)].sort('l1', ascending=False)

Non-Zero,RSS_train,RSS_valid,l1
10,1.11806872494e+15,9.66925692362e+14,2976351441.63
12,1.00925453308e+15,8.69018172894e+14,2335721469.09
13,9.28320591248e+14,7.9616310964e+14,1832980710.83
15,8.61730552584e+14,7.37850622829e+14,1438449888.29
15,8.18922951373e+14,7.01046815867e+14,1128837891.68
16,7.90904135593e+14,6.77261520728e+14,885866790.41
17,7.71729684385e+14,6.60962217696e+14,695192796.178
17,7.57956468633e+14,6.48983455376e+14,545559478.117
17,7.49031998022e+14,6.41261198311e+14,428133239.872
17,7.43226365508e+14,6.3626814023e+14,335981828.628
